# Исследование надёжности заёмщиков

**Описание проекта**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Оглавление:**


[Шаг 1. Загрузка данных](#step_1)


[Шаг 2. Переработка данных](#step_2)
+ [Шаг 2.1. Обработка пропусков и изменение типа данных](#step_2_1)
+ [Шаг 2.2. Замена типа данных](#step_2_2)
+ [Шаг 2.3. Удаление дубликатов](#step_2_3)
+ [Шаг 2.4. Лемматизация](#step_2_4)
+ [Шаг 2.5. Категоризация](#step_2_5)

[Шаг 3. Ответьте на вопроса](#step_3)

[Шаг 3.1. Есть ли зависимость между наличием детей и возвратом кредита в срок?](#step_3_1)
+ [Шаг 3.2. Есть ли зависимость между семейным положением и возвратом кредита в срок?](#step_3_2)
+ [Шаг 3.3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#step_3_3)
+ [Шаг 3.4. Как разные цели кредита влияют на его возврат в срок?](#step_3_4)

[Шаг 4. Общий вывод](#step_4)

<a id='step_1'></a>
**Шаг №1. Загрузим данные**

In [2]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')

#посмотрим общую информаци о данных и первые и послендние 5 строк

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [518]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод:**

1. неверный формат данных: days_employed -> int, total_income -> int
2. есть пропуски: у нас 2к+(10%) пропусков в позициях трудовой стаж и общих доход, что в принципе не мало.
3. на первый взгляд: минусовой трудовой стаж в днях, дублирование слов в поле purpose: на покупку своего автомобиля или просто автомобиля или все таки приобритение автомобиля, Среднее или СРЕДНЕЕ?

<a id='step_2'></a>
**Шаг №2. Переработка данных**

<a id='step_2_1'></a>
**Шаг №2.1. Обработка пропусков и изменение типа данных**

In [519]:
#посмотрим первые 5 строк данных, где total_income имеет пропуски

data[data['total_income'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [520]:
# посчитаем для пустых в total_income непустые значения в других стролбцах

data[data['total_income'].isnull()].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

In [521]:
# сначала приведем в порядом значения категориальных параметров
# посмотрим на значения поля образование

data.groupby(by = 'education')['debt'].count()

education
ВЫСШЕЕ                   274
Высшее                   268
НАЧАЛЬНОЕ                 17
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
Начальное                 15
Неоконченное высшее       47
СРЕДНЕЕ                  772
Среднее                  711
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
высшее                  4718
начальное                250
неоконченное высшее      668
среднее                13750
ученая степень             4
Name: debt, dtype: int64

In [522]:
#преведем все к нижнему регистру
data['education'] = data['education'].str.lower()

In [523]:
#посмотрим на все возможные должности
data.groupby(by = 'income_type')['debt'].agg({'count', 'sum'})

#здесь все хорого

,count,sum
income_type,,
безработный,2,1
в декрете,1,1
госслужащий,1459,86
компаньон,5085,376
пенсионер,3856,216
предприниматель,2,0
сотрудник,11119,1061
студент,1,0


In [524]:
# посмотрим на значения поля семейный статус

data.groupby(by = 'family_status')['debt'].agg({'count', 'sum'})

,count,sum
family_status,,
Не женат / не замужем,2813,274
в разводе,1195,85
вдовец / вдова,960,63
гражданский брак,4177,388
женат / замужем,12380,931


In [525]:
#преведем все к нижнему регистру
data['family_status'] = data['family_status'].str.lower()

In [526]:
# посмотрим на возможный пол
data.groupby(by = 'gender')['debt'].agg({'count', 'sum'})

,count,sum
gender,,
F,14236,994
M,7288,747
XNA,1,0


In [527]:
# какой-то странный пол XNA. Так как всего лишь 1 человек, удалим этого клиента из выборки. 
data.loc[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [528]:
data = data.drop([10701]).reset_index(drop=True)
data.groupby(by = 'gender')['debt'].count()

gender
F    14236
M     7288
Name: debt, dtype: int64

In [529]:
#посмотрим на кол-во детей

data.groupby(by = 'children')['debt'].agg({'count', 'sum'})

,count,sum
children,,
-1,47,1
0,14148,1063
1,4818,444
2,2055,194
3,330,27
4,41,4
5,9,0
20,76,8


In [530]:
#у нас есть миносовые дети. Верноятно, это тире. Переведем отрицательные значения в положительные
data['children'] = data['children'].abs()

In [531]:
#у нас так же есть миносовые значения в трудовом стаже. 
#Предположим, что это тоже оопечатка в виде тире и переведем все отрицательные значения в положительные

data['days_employed'] = data['days_employed'].abs()

**ВЫВОД:**

В позициях дети и общий трудовой стаж есть миносовые значения. Вероятно, это оппечатки в виде тире. Мы заменили их на положительрные значения. Нашли странный пол XNA и удалили его. Также привели к нижнем регистру категориальные значени: образование и семейный статус. 
В данных есть 2174 пропуска в значения трудовой стаж и уровень дохода обновременно. Изучим эти данные и посмотрим, что с ними можно сделать. 

In [532]:
# посмотрим у скольких была задолженность по кредиту для выборки с пустыми значения в доходе.

nan_income_debt = data[data['total_income'].isnull()].groupby(by = 'debt')['debt'].count()
nan_income_debt

debt
0    2004
1     170
Name: debt, dtype: int64

In [533]:
#а вообще у скольких была задоженность по кредиту?

income_debt = data.groupby(by = 'debt')['debt'].count()
print(income_debt)
print()

debt
0    19783
1     1741
Name: debt, dtype: int64



In [ ]:
#посчитаем процентое соотношение
print('% клиентов, у которых была задолженность по кредиту: {:.3%}'.format(income_debt[1]/income_debt[0]), '\n% клиетов с неизвестным доходом, у которых была задолженность по кредиту {:.3%}'.format(nan_income_debt[1]/nan_income_debt[0]))
print()

In [535]:
# посмотрим в каких должностях работают клиенты, по которым нам не известен среднемесячный доход
print(data[data['total_income'].isnull()].groupby(by = 'income_type')['debt'].agg({'count', 'sum'}))

                 count  sum
income_type                
госслужащий        147    7
компаньон          508   30
пенсионер          413   35
предприниматель      1    0
сотрудник         1105   98


In [536]:
# посмотрим на образование клиентов, по которым нам не известен среднемесячный доход
print(data[data['total_income'].isnull()].groupby(by = 'education')['debt'].agg({'count', 'sum'}))

                     count  sum
education                      
высшее                 544   26
начальное               21    3
неоконченное высшее     69    3
среднее               1540  138


In [537]:
# посмотрим на наличие детей у клиентов, по которым неизвестен уровень дохода
print(data[data['total_income'].isnull()].groupby(by = 'children')['debt'].agg({'count', 'sum'}))

          count  sum
children            
0          1439  111
1           478   36
2           204   17
3            36    5
4             7    1
5             1    0
20            9    0


In [538]:
# посмотрим на семейное положение клиентов, по которым неизвестен уровень дохода
print(data[data['total_income'].isnull()].groupby(by = 'family_status')['debt'].agg({'count', 'sum'}))

                       count  sum
family_status                    
в разводе                112    9
вдовец / вдова            95    7
гражданский брак         442   49
женат / замужем         1237   85
не женат / не замужем    288   20


In [539]:
# для того, чтобы посмотреть возрастные группы клиентов, по которым мы не знаем уровень ухода
# сделаем возрастные группы: до 20, 20-25, 25-30, 30-40, 40-50, 50-60, старше 60 (последнее значение не включено)

def age_group(age):
    if age < 20: return 'до 20 лет'
    if 20 <= age < 25: return '20-25 лет'
    if 25 <= age < 30: return '25-30 лет'
    if 30 <= age < 40: return '30-40 лет'
    if 40 <= age < 50: return '40-50 лет'
    if 50 <= age < 60: return '50-60 лет'
    else: return 'старше 60 лет'

       
data['age_group'] = data['dob_years'].apply(age_group)

data[data['total_income'].isnull()].groupby(by = 'age_group')['debt'].agg({'count', 'sum'})

,count,sum
age_group,,
20-25 лет,97,8
25-30 лет,201,23
30-40 лет,565,55
40-50 лет,537,35
50-60 лет,501,35
до 20 лет,11,0
старше 60 лет,262,14


In [540]:
# посмотрим, как вообще расредлеяется поведение клиентов по возратам
data.groupby(by = 'age_group')['debt'].agg({'count', 'sum'})

,count,sum
age_group,,
20-25 лет,862,87
25-30 лет,2306,261
30-40 лет,5674,552
40-50 лет,5371,404
50-60 лет,4679,305
до 20 лет,115,9
старше 60 лет,2517,123


**ВЫВОД:** 

У нас 1 предприниматель (из 2х). Его мы удалим, так как он один, а всего их два. Еще можно было бы удалить клиентов с неизвестным доходом в возрасте мледше 20 лет, но попробуем востановить данные, так как доля клиентов с неизвестным доходом составляет 8,8%. 
Посмотрим, по каким параментрам мы можем востановить данные. 

In [541]:
# удалим 1 препринемателя
data.loc[data['income_type'] == 'предприниматель']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости,50-60 лет
18696,0,520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы,25-30 лет


In [542]:
data = data.drop([5936]).reset_index(drop=True)

In [543]:
# теперь посмотрим как влият ли на уровень дохода семейный статус, наличией детей и должность. По отдельности и в сумме. 
# для этого сочитаем откломение медаины и средней по группе от общей по данным. 

data_no_null = data.dropna()

data_no_null['total_income']=data_no_null['total_income'].astype(int)
#data_no_null_mean = data_no_null['total_income'].mean()
#data_no_null_median = data_no_null['total_income'].median()

data_no_null_family = data_no_null.pivot_table(index = 'family_status', values = 'total_income', aggfunc = {'median', 'mean'})
#data_no_null_family['mean'] = data_no_null_family['mean'] - data_no_null_mean
#data_no_null_family['median'] = data_no_null_family['median'] - data_no_null_mean
print(data_no_null_family)

data_no_null_age = data_no_null.pivot_table(index = 'age_group', values = 'total_income', aggfunc = {'median', 'mean'})
#data_no_null_age['mean'] = data_no_null_age['mean'] - data_no_null_mean
#data_no_null_age['median'] = data_no_null_age['median'] - data_no_null_mean
print(data_no_null_age)

data_no_null_education = data_no_null.pivot_table(index = 'education', values = 'total_income', aggfunc = {'median', 'mean'})
#data_no_null_education['mean'] = data_no_null_education['mean'] - data_no_null_mean
#data_no_null_education['median'] = data_no_null_education['median'] - data_no_null_mean
print(data_no_null_education)

print()
data_no_null_income = data_no_null.pivot_table(index = 'income_type', values = 'total_income', aggfunc = {'median', 'mean'})
#data_no_null_income['mean'] = data_no_null_income['mean'] - data_no_null_mean
#data_no_null_income['median'] = data_no_null_income['median'] - data_no_null_mean
print(data_no_null_income)

print()
print('Средний ежемесячный доход:', data_no_null_mean, '\nМедиана ежемесячного дохода:', data_no_null_median)


                                mean    median
family_status                                 
в разводе              169932.982456  146969.0
вдовец / вдова         143650.797688  128213.0
гражданский брак       166829.750402  144908.5
женат / замужем        169010.653504  146184.0
не женат / не замужем  168337.433663  144681.0
                        mean    median
age_group                             
20-25 лет      142420.783007  128912.0
25-30 лет      166133.808076  147097.0
30-40 лет      176952.500489  154172.0
40-50 лет      178445.587712  154776.0
50-60 лет      161322.627812  138769.0
до 20 лет      151821.740385  131874.5
старше 60 лет  143884.753437  123508.0
                              mean    median
education                                   
высшее               207142.009118  175340.0
начальное            132155.022989  117137.0
неоконченное высшее  181500.348665  160054.5
среднее              153715.144234  136478.0
ученая степень       174749.833333  157259.5

    

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [315]:
print(data_no_null.pivot_table(index = ['income_type', 'age_group'], values = 'total_income', aggfunc = {'median', 'mean', 'count'}))

                                count           mean    median
income_type     age_group                                     
безработный     30-40 лет         1.0   59956.000000   59956.0
                40-50 лет         1.0  202722.000000  202722.0
в декрете       30-40 лет         1.0   53829.000000   53829.0
госслужащий     20-25 лет        42.0  148879.452381  140541.5
                25-30 лет       171.0  159670.923977  148990.0
                30-40 лет       409.0  174510.982885  154142.0
                40-50 лет       399.0  178551.177945  155567.0
                50-60 лет       230.0  161487.652174  144601.0
                до 20 лет         7.0  136803.428571  114599.0
                старше 60 лет    54.0  184158.333333  149734.5
компаньон       20-25 лет       235.0  160200.702128  140402.0
                25-30 лет       613.0  187276.402936  167366.0
                30-40 лет      1474.0  207161.689959  179784.5
                40-50 лет      1326.0  212435.984917  1

В данных есть пенсионеры в каждой возрастной группе. Хотя основная масса и приходится на больше 50 лет, есть и пенсионеры в возрасте до 50. Странные данные. Можно сказать, что уровень дохода зависит от образования и должности: тут наиболее сильный разброс по группе. Но посчитаем корреляцию, для этого создадим income_type_id, age_group_id и gender_id.

In [544]:
def income_type_id(income_type):
    if income_type == 'безработный': return '1'
    if income_type == 'в декрете': return '2'
    if income_type == 'госслужащий': return '3'
    if income_type == 'компаньон': return '4'
    if income_type == 'пенсионер': return '5'
    if income_type == 'предприниматель': return '6'
    if income_type == 'студент': return '7'
    if income_type == 'сотрудник': return '8'
    
def gender_id(gender):
    if gender == 'F': return '1'
    else: return '0'
    
    
def age_group_id(age):
    if age < 20: return '1'
    if 20 <= age < 25: return '2'
    if 25 <= age < 30: return '3'
    if 30 <= age < 40: return '4'
    if 40 <= age < 50: return '5'
    if 50 <= age < 60: return '6'
    else: return '7'
    
data['age_group_id'] = data['dob_years'].apply(age_group_id)
data['age_group_id'] = data['age_group_id'].astype('int')    
data_no_null['income_type_id'] = data_no_null['income_type'].apply(income_type_id)
data_no_null['income_type_id'] = data_no_null['income_type_id'].astype('int')
data_no_null['gender_id'] = data_no_null['gender'].apply(gender_id)
data_no_null['gender_id'] = data_no_null['gender_id'].astype('int')


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

In [545]:
data_no_null.corr(method='spearman')

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,income_type_id,gender_id
children,1.000000,-0.192625,-0.338652,-0.035636,-0.157071,0.030977,0.031805,0.062102,-0.045979
days_employed,-0.192625,1.000000,0.545323,0.044909,-0.042527,-0.082058,-0.083253,-0.154208,0.157400
dob_years,-0.338652,0.545323,1.000000,0.074187,-0.045637,-0.073357,-0.076383,-0.110939,0.132394
education_id,-0.035636,0.044909,0.074187,1.000000,0.015184,0.056578,-0.188447,0.107456,-0.024623
family_status_id,-0.157071,-0.042527,-0.045637,0.015184,1.000000,0.023246,-0.014393,-0.004920,0.064367
debt,0.030977,-0.082058,-0.073357,0.056578,0.023246,1.000000,-0.008820,0.047595,-0.056009
total_income,0.031805,-0.083253,-0.076383,-0.188447,-0.014393,-0.008820,1.000000,-0.096773,-0.211195
income_type_id,0.062102,-0.154208,-0.110939,0.107456,-0.004920,0.047595,-0.096773,1.000000,-0.071902
gender_id,-0.045979,0.157400,0.132394,-0.024623,0.064367,-0.056009,-0.211195,-0.071902,1.000000


In [546]:
data_no_null.corr()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,income_type_id,gender_id
children,1.000000,-0.139842,-0.177569,-0.010224,-0.098117,0.020664,0.017814,0.051168,-0.028347
days_employed,-0.139842,1.000000,0.589855,0.080195,0.003650,-0.049087,-0.135857,-0.283070,0.154788
dob_years,-0.177569,0.589855,1.000000,0.065163,-0.070376,-0.071479,-0.052886,-0.169990,0.128814
education_id,-0.010224,0.080195,0.065163,1.000000,0.010504,0.052744,-0.178946,0.086004,-0.029159
family_status_id,-0.098117,0.003650,-0.070376,0.010504,1.000000,0.022323,-0.009148,-0.005922,0.036828
debt,0.020664,-0.049087,-0.071479,0.052744,0.022323,1.000000,-0.012470,0.051043,-0.056009
total_income,0.017814,-0.135857,-0.052886,-0.178946,-0.009148,-0.012470,1.000000,-0.090382,-0.179873
income_type_id,0.051168,-0.283070,-0.169990,0.086004,-0.005922,0.051043,-0.090382,1.000000,-0.085006
gender_id,-0.028347,0.154788,0.128814,-0.029159,0.036828,-0.056009,-0.179873,-0.085006,1.000000


**ВЫВОД:**

По анализу показателя корреляции можно сказать, что уровень дохода зависит от пола, должности, образования и стажа. Оставим пол и образование, как наиболее высокие. Но тем не менее для более граммотного заполнения уровня дохода, все таки нужно строить мат. модель по имеющимся данным. 

In [547]:
data_no_null_education_gender = data_no_null.pivot_table(index = ['education', 'gender'], values = 'total_income', aggfunc = 'mean')
print(data_no_null_education_gender)
#print(data_no_null_education_gender.loc['высшее', 'F']['total_income'])


                             total_income
education           gender               
высшее              F       189414.756459
                    M       243631.178988
начальное           F       119490.006757
                    M       148742.831858
неоконченное высшее F       165438.973621
                    M       207561.023346
среднее             F       141693.876484
                    M       176851.334686
ученая степень      F       183408.500000
                    M       170420.500000


In [548]:
print(data[data['total_income'].isnull()].pivot_table(index = ['education', 'gender'], values = 'debt', aggfunc = {'sum', 'count'}))

#print(data[data['total_income'].isnull()].groupby(by = 'gender')['debt'].count())


                            count  sum
education           gender            
высшее              F         373   18
                    M         170    8
начальное           F          15    2
                    M           6    1
неоконченное высшее F          43    2
                    M          26    1
среднее             F        1053   77
                    M         487   61


In [549]:
#заменим пустые значение в позиции трудовой стаж средним значением по всей выборке 
#(так как мы не используем данный параметр для анализа)

days_employed_mean = data['days_employed'].mean()

def fill_na(row):
    gender = row['gender']
    total_income = row['total_income']
    education = row['education']
    if total_income != total_income:
        return data_no_null_education_gender.loc[education, gender]['total_income']
    else: return total_income
    
data['total_income'] = data.apply(fill_na, axis =1)

In [550]:
# проверим себя
data['total_income'].isna().sum()

0

In [551]:
#заменим пустые значение в позиции трудовой стаж средним значением по всей выборке 
#(так как мы не используем данный параметр для анализа)

days_employed_mean = data['days_employed'].mean()

def fill_na_days(days_employed):
    if days_employed != days_employed: 
        return days_employed_mean
    else: return days_employed
    
data['days_employed'] = data['days_employed'].apply(fill_na_days)
    


In [499]:
# проверим себя 
data['days_employed'].isna().sum()

0

**ВЫВОД:** 

Итого в данные 2174 клиентов, по которым неизвестен трудовой стаж и ежемесячный доход. 
Это госслужащие, компаньоны, пансионеры, сотрудники и 1 предприниматель и в основном с высшим и средним образованием. Так как в данных всего препринимателей 2, мы удалим одного, от них данные не поменяются. 

Для заполнения пропусков по критерию ежемесячный доход провели анализ взаимосвязи дохода от других показателей:  образованию, должности, семейному статусу и возрастной группе. Так как линейно яркой взаимосвязи не видно, посчитали корреляцию и выбрали показатели должность и пол как наиболее зависимые показатели. По ним посчитали средний доход по группе и заполнили пустые значения в искомой выборке. 

Пустые значения в трудовом стаже заполнили средней по общей выборке, так как данный параметр мы не анализируем. 

<a id='step_2_2'></a>
**Шаг №2.2. Замена типа данных**

In [552]:
# заменим еще раз тип данных на int для total_income и days_employed

data['total_income']=data['total_income'].astype(int)
data['days_employed']=data['days_employed'].astype(int)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21523 entries, 0 to 21522
Data columns (total 14 columns):
children            21523 non-null int64
days_employed       21523 non-null int64
dob_years           21523 non-null int64
education           21523 non-null object
education_id        21523 non-null int64
family_status       21523 non-null object
family_status_id    21523 non-null int64
gender              21523 non-null object
income_type         21523 non-null object
debt                21523 non-null int64
total_income        21523 non-null int64
purpose             21523 non-null object
age_group           21523 non-null object
age_group_id        21523 non-null int64
dtypes: int64(8), object(6)
memory usage: 2.3+ MB
None


**ВЫВОД:** Поменяем тип данных dob_years и total_income на целочисленный, так как эти показатели должны быть целыми.

<a id='step_2_3'></a>
**Шаг №2.3. Обработка дубликатов**

In [553]:
# посмотрим на варианты целей кредита. 
print(data.groupby('purpose')['purpose'].count())

purpose
автомобили                                478
автомобиль                                495
высшее образование                        453
дополнительное образование                462
жилье                                     647
заняться высшим образованием              496
заняться образованием                     412
на покупку автомобиля                     472
на покупку подержанного автомобиля        479
на покупку своего автомобиля              505
на проведение свадьбы                     777
недвижимость                              634
образование                               447
операции с жильем                         653
операции с коммерческой недвижимостью     651
операции с недвижимостью                  676
операции со своей недвижимостью           630
покупка жилой недвижимости                606
покупка жилья                             647
покупка жилья для сдачи                   653
покупка жилья для семьи                   641
покупка коммерческой недви

In [554]:
# в целях кредита творится хаос. и здесь нужна будем лемматизация.
# пока удалим явно забудлированные строки и проверим себя.

data.duplicated().sum()

71

In [555]:
data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum()

0

**ВЫВОД:** Нашли 71 дублкат. Предположим, что они появились в следствии тех. багов при загрузки данных. Удалил их.

<a id='step_2_4'></a>
**Шаг №2.4. Лемматизация**


In [3]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

#создадим список уникальных целей для получения кредита.

purpose = str(data['purpose'].unique())
print(purpose)

#лематизируем и посчитаем чаще встречающиеся значения
lemas = m.lemmatize(purpose)
print(Counter(lemas))
  
#from nltk.stem import SnowballStemmer
#russian_stemer = SnowballStemmer('russian')
#print(russian_stemer.stem(purpose))

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']
Counter({' ': 59, "' '": 23, 

Чаще встречающиеся слова в целях получения кредита: покупка, недвижимость, автомобиль, образование, жилье, свадьба, строительство. Заполним тэги по этим словам.

In [557]:
def purpose_group(purpose):
    purpose_dic = ['недвижимость','автомобиль', 'образование', 'жилье', 'свадьба']
    for purp in purpose_dic: 
        if purp in m.lemmatize(purpose):
            return purp
      
data['tag'] = data['purpose'].apply(purpose_group)  
data.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,age_group_id,tag
21447,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,40-50 лет,5,жилье
21448,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,старше 60 лет,7,автомобиль
21449,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,30-40 лет,4,недвижимость
21450,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,30-40 лет,4,автомобиль
21451,2,1984,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047,на покупку автомобиля,40-50 лет,5,автомобиль


In [558]:
#теперь заменим жилье на недвижимость. все таки это одна категория, и недвижимость более подходит. 

data['tag'] = data['tag'].replace('жилье', 'недвижимость')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,age_group_id,tag
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,40-50 лет,5,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,30-40 лет,4,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,30-40 лет,4,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,30-40 лет,4,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,50-60 лет,6,свадьба


### Вывод

**ВЫВОД:** Чаще встречающиеся слова в целях получения кредита: недвижимость, автомобиль, образование, жилье, свадьба. Заполним тэги по этим словам, а потом заменили жилье на недвижимость, так как это одна категория.

<a id='step_2_5'></a>
**Шаг №2.5. Категоризация данных**

In [559]:
#мы уже считали среднюю и медиану. возьмем шаг 50 тыс., так как с примерно шагом как раз и идут 25, 50 и 75%

def income_type(income):
    if income <= 50000: 
        return 'до 50к'
    if 50000 < income < 100000:
        return '50-100к'
    if 100000 <= income <= 150000:
        return '100-150к'
    if 150000 < income < 200000:
        return '150-200к'
    else: return 'больше 200'
    
data['income_group'] = data['total_income'].apply(income_type)
data_no_null['income_group'] = data_no_null['total_income'].apply(income_type)

#data_total_income_grouped = data_total_income.pivot_table(index = 'income_group', values = 'debt', aggfunc = ['sum', 'count'])
#print(data_total_income_grouped)


In [562]:
#сделаем категорию дети: есть/нет

def child(children):
    if children == 0: 
        return 'детей нет'
    if children == 1:
        return '1 ребенок'
    if children == 2:
        return '2 ребенка'
    else: return '3 и более детей'


data['child_group'] = data['children'].apply(child)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,age_group_id,tag,income_group,child_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,40-50 лет,5,недвижимость,больше 200,1 ребенок
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,30-40 лет,4,автомобиль,100-150к,1 ребенок
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,30-40 лет,4,недвижимость,100-150к,детей нет
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,30-40 лет,4,образование,больше 200,3 и более детей
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,50-60 лет,6,свадьба,150-200к,детей нет


**ВЫВОД:** Создали 2 категории: наличие детей и уровень дохода для удобства анализа данных.

<a id='step_3'></a>
**Шаг №3. Ответьте на вопросы**

<a id='step_3_1'></a>
**Шаг №3.1. Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [563]:
child_pivot_table = data.pivot_table(index = 'child_group', columns = 'debt', values = 'dob_years', aggfunc = 'count')

child_pivot_table_per = 100 * child_pivot_table / data.shape[0]
child_pivot_table_per ['total %'] = child_pivot_table_per[0] + child_pivot_table_per[1]
child_pivot_table['% вернуших'] = 100 * child_pivot_table[0]/(child_pivot_table[0]+child_pivot_table[1])
#child_pivot_table.loc['%'] = [100*child_pivot_table.loc['детей нет', 0]/child_pivot_table.loc['дети есть', 0], 100*child_pivot_table.loc['детей нет', 1]/child_pivot_table.loc['дети есть', 1], '']

# соотношение заложенности по кредиту относительно показателя с детьми/без детей, в человеках, в %
print(child_pivot_table.sort_values(by = '% вернуших', ascending  = False))
print()
print(child_pivot_table_per.sort_values(by = 'total %', ascending  = False))

debt                 0     1  % вернуших
child_group                             
детей нет        13026  1063   92.455107
3 и более детей    417    39   91.447368
1 ребенок         4410   445   90.834192
2 ребенка         1858   194   90.545809

debt                     0         1    total %
child_group                                    
детей нет        60.721611  4.955249  65.676860
1 ребенок        20.557524  2.074399  22.631922
2 ребенка         8.661197  0.904345   9.565542
3 и более детей   1.943875  0.181801   2.125676


**ВЫВОД:** Клиенты без детей чаще берут кредит и лучше его возращают: 65% клинетов банка - люди без детей и 92,45% их них не имели задолженности по кредиту. Клиенты с детьми и реже берут кредиты, и хуже их отдают. Тем не менее можно однозначно сказать, что многодетные семьи лучше отдают кредиты: % вернувших составляет 91,44%. 

<a id='step_3_2'></a>
**Шаг №3.2. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [564]:
family_pivot_table=data.pivot_table(index = 'family_status', columns = 'debt', values = 'dob_years', aggfunc = 'count')

family_pivot_table_per = 100 * family_pivot_table / data.shape[0]
family_pivot_table_per ['total %'] = family_pivot_table_per[0] + family_pivot_table_per[1]
family_pivot_table['% вернуших'] = 100 * family_pivot_table[0]/(family_pivot_table[0]+family_pivot_table[1])
family_pivot_table['% невернуших'] = 100 * family_pivot_table[1]/(family_pivot_table[0]+family_pivot_table[1])

# соотношение заложенности по кредиту относительно показателя по семейному статусу, в человеках, в %
print(family_pivot_table.sort_values(by = '% вернуших', ascending  = False))
print()
print(family_pivot_table_per.sort_values(by = 'total %', ascending  = False))

debt                       0    1  % вернуших  % невернуших
family_status                                              
вдовец / вдова           896   63   93.430657      6.569343
в разводе               1110   85   92.887029      7.112971
женат / замужем        11407  931   92.454207      7.545793
гражданский брак        3762  388   90.650602      9.349398
не женат / не замужем   2536  274   90.249110      9.750890

debt                           0         1    total %
family_status                                        
женат / замужем        53.174529  4.339922  57.514451
гражданский брак       17.536826  1.808689  19.345516
не женат / не замужем  11.821742  1.277270  13.099012
в разводе               5.174343  0.396233   5.570576
вдовец / вдова          4.176767  0.293679   4.470446


**ВЫВОД:** Самая надежная группа это вдовцы/вдовы, но тем неменее это всего 4,5% всех клиентов банка. Следующие по надежности идут люди в разводе с 92,88% не имеющих задолженность по кредиту, но и они составляют всего 5,57% всех клиентов банка. А вот женатые/замужнее чаще всех берут кредит - это 57% клиентов банка, но тем не менее % возрата близок к показателю людей в разводе и составляет 92,45%. Самая же ненадежная группа клиентов - это одинокие люди и пары в гражданском браке, но тем не менее эта группа составляет более 30% всех клиентов банка. 

<a id='step_3_3'></a>
**Шаг №3.3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [565]:
#data_income_grouped = data_total_income.groupby(['income_group']).agg({'debt': ['sum', 'count']})
#data_income_grouped['%'] = 100 * data_income_grouped['debt']['sum']/data_income_grouped['debt']['count'].sum()
#print(data_income_grouped)

income_pivot_table=data.pivot_table(index = 'income_group', columns = 'debt', values = 'dob_years', aggfunc = 'count')

income_pivot_table_per = 100 * income_pivot_table / data.shape[0]
income_pivot_table_per ['total %'] = income_pivot_table_per[0] + income_pivot_table_per[1]
income_pivot_table['% вернуших'] = 100 * income_pivot_table[0]/(income_pivot_table[0]+income_pivot_table[1])
income_pivot_table['% невернуших'] = 100 * income_pivot_table[1]/(income_pivot_table[0]+income_pivot_table[1])


# соотношение заложенности по кредиту относительно показателя по уровню дохода, в человеках, в %
print(income_pivot_table.sort_values(by = '% вернуших', ascending  = False))
print()
print(income_pivot_table_per.sort_values(by = 'total %', ascending  = False))

debt             0    1  % вернуших  % невернуших
income_group                                     
до 50к         349   23   93.817204      6.182796
больше 200    4892  367   93.021487      6.978513
50-100к       3760  331   91.909069      8.090931
100-150к      6153  571   91.508031      8.491969
150-200к      4557  449   91.030763      8.969237

debt                  0         1    total %
income_group                                
100-150к      28.682640  2.661756  31.344397
больше 200    22.804401  1.710796  24.515197
150-200к      21.242775  2.093045  23.335820
50-100к       17.527503  1.542980  19.070483
до 50к         1.626888  0.107216   1.734104


In [566]:
# изучем поведение клиенток по выборке без замененых неизвестных данных по уровню дохода
income_null_pivot_table=data_no_null.pivot_table(index = 'income_group', columns = 'debt', values = 'dob_years', aggfunc = 'count')

income_null_pivot_table_per = 100 * income_null_pivot_table / data.shape[0]
income_null_pivot_table_per ['total %'] = income_null_pivot_table_per[0] + income_null_pivot_table_per[1]
income_null_pivot_table['% вернуших'] = 100 * income_null_pivot_table[0]/(income_null_pivot_table[0]+income_null_pivot_table[1])
income_null_pivot_table['% невернуших'] = 100 * income_null_pivot_table[1]/(income_null_pivot_table[0]+income_null_pivot_table[1])


# соотношение заложенности по кредиту относительно показателя по уровню дохода, в человеках, в %
print(income_null_pivot_table.sort_values(by = '% вернуших', ascending  = False))
print()
print(income_null_pivot_table_per.sort_values(by = 'total %', ascending  = False))

debt             0    1  % вернуших  % невернуших
income_group                                     
до 50к         349   23   93.817204      6.182796
больше 200    4707  358   92.931885      7.068115
50-100к       3760  331   91.909069      8.090931
100-150к      5213  491   91.392006      8.607994
150-200к      3750  368   91.063623      8.936377

debt                  0         1    total %
income_group                                
100-150к      24.300764  2.288831  26.589595
больше 200    21.942010  1.668842  23.610852
150-200к      17.480888  1.715458  19.196345
50-100к       17.527503  1.542980  19.070483
до 50к         1.626888  0.107216   1.734104


**ВЫВОД:** Рассматривая уровень дохода, можно сказать, что люди с доходом менее 50к реже всего берут кредит. Остальные группы проявляют активность примерно на одном уровне. Люди с доходом менее 50к и больше 200к лучше всего возращают кредит: более 93% не имели задолженности по кредиты. % вернувший кредит в срок по остальным группам примерно одинаков и составляет 91-92%.

<a id='step_3_4'></a>
**Шаг №3.4. Как разные цели кредита влияют на его возврат в срок?**

In [351]:
#data_purpose_grouped = data.groupby('tag')['debt'].agg({'debt': ['sum', 'count']})
#data_purpose_grouped['%'] = 100 * data_purpose_grouped['debt']['sum']/data_purpose_grouped['debt']['count'].sum()
#print(data_purpose_grouped)

purpose_pivot_table = data.pivot_table(index = 'tag', columns = 'debt', values = 'dob_years', aggfunc = 'count')

purpose_pivot_table_per = 100 * purpose_pivot_table / data.shape[0]
purpose_pivot_table_per ['total %'] = purpose_pivot_table_per[0] + purpose_pivot_table_per[1]
purpose_pivot_table['% вернуших'] = 100 * purpose_pivot_table[0]/(purpose_pivot_table[0]+purpose_pivot_table[1])
purpose_pivot_table['% невернуших'] = 100 * purpose_pivot_table[1]/(purpose_pivot_table[0]+purpose_pivot_table[1])
                                                                    
print(purpose_pivot_table.sort_values(by = '% вернуших', ascending  = False))
print()
print(purpose_pivot_table_per.sort_values(by = 'total %', ascending  = False))

debt              0    1  % вернуших  % невернуших
tag                                               
недвижимость  10027  782   92.765288      7.234712
свадьба        2138  186   91.996558      8.003442
образование    3643  370   90.779965      9.220035
автомобиль     3903  403   90.640966      9.359034

debt                  0         1    total %
tag                                         
недвижимость  46.741563  3.645348  50.386910
автомобиль    18.194108  1.878613  20.072720
образование   16.982100  1.724781  18.706880
свадьба        9.966437  0.867052  10.833489


**ВЫВОД:** Здесь можно однозначно сказать, клиенты, которы берут кредит с целью операций с недвижимостью, являются самыми благонадежными: 92,76% таких клиентов не имели задолженности по кредиту. Здесь можно отметить, что банк идет в правильном направлении в маркетинговой политике, так как такие клиенты составляют 50% всех клиентов банка. Самая неблагонадежные цели - это кредит на образование или операции с автомобилем. 9,3% таких клиентов когда-либо имели задолженность по кредиту. Группа клиентов, которые берут кредит на организацию свадьбы, является нейтральной - 8% таких клиентов, но это и самая небольшая группа людей, всего лишь 10% таких клиентов обращаются в банк. 

<a id='step_4'></a>
**Шаг №4. Общий вывод**

In [376]:
# посмотрим как возрат влият а возрат кредита в срок
age_pivot_table = data.pivot_table(index = 'age_group', columns = 'debt', values = 'dob_years', aggfunc = 'count')

age_pivot_table_per = 100 * age_pivot_table / data.shape[0]
age_pivot_table_per ['total %'] = age_pivot_table_per[0] + age_pivot_table_per[1]
age_pivot_table['% вернуших'] = 100 * age_pivot_table[0]/(age_pivot_table[0]+age_pivot_table[1])
age_pivot_table['% невернуших'] = 100 * age_pivot_table[1]/(age_pivot_table[0]+age_pivot_table[1])
                                                                    
print(age_pivot_table.sort_values(by = '% вернуших', ascending  = False))
print()
print(age_pivot_table_per.sort_values(by = 'total %', ascending  = False))

debt              0    1  % вернуших  % невернуших
age_group                                         
старше 60 лет  2377  123   95.080000      4.920000
50-60 лет      4351  305   93.449313      6.550687
40-50 лет      4950  404   92.454240      7.545760
до 20 лет       106    9   92.173913      7.826087
30-40 лет      5110  552   90.250795      9.749205
20-25 лет       773   87   89.883721     10.116279
25-30 лет      2044  261   88.676790     11.323210

debt                   0         1    total %
age_group                                    
30-40 лет      23.820623  2.573187  26.393809
40-50 лет      23.074772  1.883274  24.958046
50-60 лет      20.282491  1.421779  21.704270
старше 60 лет  11.080552  0.573373  11.653925
25-30 лет       9.528249  1.216670  10.744919
20-25 лет       3.603394  0.405557   4.008950
до 20 лет       0.494126  0.041954   0.536081


In [374]:
debt_family_child = data.pivot_table(index = ['family_status', 'child_group'], columns = 'debt', values = 'dob_years', aggfunc = 'count')
debt_family_child ['% вернувших'] = 100 * debt_family_child[0]/(debt_family_child[0]+debt_family_child[1])
print(debt_family_child.sort_values(by = '% вернувших', ascending  = False).tail(10))
print()
debt_family_purpose = data.pivot_table(index = ['family_status', 'tag'], columns = 'debt', values = 'dob_years', aggfunc = 'count')
debt_family_purpose ['% вернувших'] = 100 * debt_family_purpose[0]/(debt_family_purpose[0]+debt_family_purpose[1])
print(debt_family_purpose.sort_values(by = '% вернувших', ascending  = False).tail(10))
print()
debt_income_tag = data.pivot_table(index = ['income_group', 'tag'], columns = 'debt', values = 'dob_years', aggfunc = 'count')
debt_income_tag ['% вернувших'] = 100 * debt_income_tag[0]/(debt_income_tag[0]+debt_income_tag[1])
print(debt_income_tag.sort_values(by = '% вернувших', ascending  = False).tail(10))
print()

debt_all = data.pivot_table(index = ['age_group', 'tag'], columns = 'debt', values = 'dob_years', aggfunc = 'count')
debt_all['% вернувших'] = 100 * debt_all[0]/(debt_all[0] + debt_all[1])
print(debt_all.sort_values(by = '% вернувших', ascending  = False))



debt                                        0      1  % вернувших
family_status         child_group                                
не женат / не замужем детей нет        2052.0  210.0    90.716180
женат / замужем       2 ребенка        1388.0  145.0    90.541422
не женат / не замужем 1 ребенок         402.0   52.0    88.546256
гражданский брак      1 ребенок         882.0  118.0    88.200000
не женат / не замужем 2 ребенка          66.0    9.0    88.000000
гражданский брак      3 и более детей    67.0   11.0    85.897436
в разводе             3 и более детей    12.0    2.0    85.714286
вдовец / вдова        2 ребенка          17.0    3.0    85.000000
не женат / не замужем 3 и более детей    16.0    3.0    84.210526
вдовец / вдова        3 и более детей    11.0    NaN          NaN

debt                                   0    1  % вернувших
family_status         tag                                 
гражданский брак      свадьба       2138  186    91.996558
не женат / не замужем недвижим

**ОБЩИЙ ВЫВОД:** 

Клиенты, у которых есть деть, с целями образование и операции с автомобилем, одинокие или в гражданском браке, чаще всего имеют задолженность по кредиту. Отсутсвие детей, цель кредита - недвижимость, положительный семейный статус повышает надежность клиента Клиенты с доходом более 200к и менее 50к реже имеют задолженность по кредиту, а уровень надежности клиентов с уровнем дохода от 50 до 200 тыс находится примерно на одном уровне и соотавляет 91-92%